In [1]:
import os
from ragas import evaluate
from datasets import Dataset
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, context_entity_recall, answer_similarity, answer_correctness
from constants import EVAL_ANSWERS, EVAL_QUESTIONS
from ragas.metrics.critique import harmfulness

In [1]:
from langchain_core.chat_history import BaseChatMessageHistory
from conversation import CustomChatMessageHistory

store = {}
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = CustomChatMessageHistory()
    return store[session_id]

In [3]:
from langchain_core.runnables.history import RunnableWithMessageHistory
from rag import caller_with_context, chain_with_context

full_chain_with_context = chain_with_context()
full_chain_with_context_and_message_history = RunnableWithMessageHistory(
    full_chain_with_context,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
)

In [4]:
sess_id = "X123"

result = full_chain_with_context_and_message_history.invoke(
        {"question": "Apa saja yang termasuk dalam Bank Pelapor?"},
        config={"configurable": {"session_id": sess_id}})

{'result': 'ketentuan_terkait', 'question': 'Apa saja yang termasuk dalam Bank Pelapor?', 'history': []}


c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  warn_beta(
Error in RootListenersTracer.on_chain_end callback: KeyError('output')


In [5]:
result

{'query': 'Apa saja yang termasuk dalam Bank Pelapor?',
 'result': 'Dalam konteks yang diberikan, tidak ada informasi yang menjelaskan mengenai apa yang termasuk dalam Bank Pelapor.',
 'source_documents': [Document(page_content='Bank yang bersangkutan”, dengan mencantumkan keterangan bahwa\npencairannya hanya dapat dilakukan setelah mendapat persetujuan\ntertulis dari Dewan Gubernur Bank Indonesia; dan\nj. surat pernyataan dari calon pemegang saham bagi Bank yang berbentuk badan hukum Perseroan Terbatas/Perusahaan Daerah atau dari calon anggota bagi Bank yang berbentuk badan hukum Koperasi, bahwa setoran modal sebagaimana dimaksud dalam huruf i:\n1. tidak berasal dari pinjaman atau fasilitas pembiayaan dalam bentuk\napapun dari Bank dan/atau pihak lain di Indonesia; dan/atau\n2. tidak berasal dari dan untuk tujuan pencucian uang (money laundering).\n(2) Daftar calon pemegang saham atau daftar calon anggota sebagaimana dimaksud pada ayat (1) huruf b:\na. dalam hal perorangan wajib diser

In [21]:
from ragas.metrics import faithfulness, answer_relevancy, context_relevancy, context_recall
from ragas.langchain import RagasEvaluatorChain

# make eval chains
eval_chains = {
    m.name: RagasEvaluatorChain(metric=m) 
    for m in [faithfulness, answer_relevancy, context_relevancy, context_recall]
}


OpenAIKeyNotFound: OpenAI API key not found! Seems like your trying to use Ragas metrics with OpenAI endpoints. Please set 'OPENAI_API_KEY' environment variable

In [ ]:
queries = EVAL_QUESTIONS
ground_truths = EVAL_ANSWERS

In [ ]:
sess_id = "X123"
results = []
contexts = []

for query in queries:
    result = full_chain_with_context_and_message_history.invoke(
        {"question": query},
        config={"configurable": {"session_id": sess_id}})
    
    print(result)
    results.append(result['answer'])
    sources = result["context"]
    contents = []
    for i in range(len(sources)):
        contents.append(sources[i][0].page_content)
    contexts.append(contents)

d = {
    "question": queries,
    "answer": results,
    "contexts": contexts,
    "ground_truth": ground_truths
}

dataset = Dataset.from_dict(d)

In [ ]:
import json

with open("dict_eval.json", "w") as outfile: 
    json.dump(d, outfile)


In [10]:
from azure_config import azure_llm, azure_embeddings
llm = azure_llm()
embeddings = azure_embeddings()


In [17]:
import json
with open('dict_eval.json') as json_file:
    d = json.load(json_file)


In [18]:
from datasets import Dataset
dataset = Dataset.from_dict(d)

In [19]:
dataset

Dataset({
    features: ['question', 'answer', 'contexts', 'ground_truths'],
    num_rows: 15
})

In [20]:
from ragas.metrics import faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness
from ragas.metrics.critique import harmfulness
from ragas import evaluate

score = evaluate(dataset, metrics=[faithfulness, answer_relevancy, context_precision, context_recall, answer_similarity, answer_correctness, harmfulness])

ValueError: Dataset feature "ground_truths" should be of type Sequence[string], got <class 'datasets.features.features.Value'>

In [ ]:
from ragas.metrics.critique import harmfulness


In [ ]:
score_df = asyncio.run(eval_run())

In [ ]:
sources[1][0].page_content

In [ ]:
sources

In [7]:
from langchain.document_loaders import WebBaseLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

# load the Wikipedia page and create index
loader = WebBaseLoader("https://en.wikipedia.org/wiki/New_York_City")
index = VectorstoreIndexCreator(embedding=embeddings).from_loaders([loader])

# create the QA chain
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=index.vectorstore.as_retriever(), return_source_documents=True
)

# testing it out
question = "How did New York City get its name?"
result = qa_chain({"query": question})
result["result"]


USER_AGENT environment variable not set, consider setting it to identify your requests.


c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain\indexes\vectorstore.py:129: UserWarning: Using InMemoryVectorStore as the default vectorstore.This memory store won't persist data. You should explicitlyspecify a vectorstore when using VectorstoreIndexCreator
  warnings.warn(
c:\Users\Bryan Delton\anaconda3\envs\ocbc_env\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'New York City got its name in 1664 when it was renamed after the Duke of York (later King James II of England). The city was originally called New Amsterdam when it was founded by Dutch colonists in 1626. However, when the English took control of the area, it was renamed New York in honor of the Duke of York, who was granted the lands by his brother, King Charles II. The name New York has remained ever since.'

In [8]:
result

{'query': 'How did New York City get its name?',
 'result': 'New York City got its name in 1664 when it was renamed after the Duke of York (later King James II of England). The city was originally called New Amsterdam when it was founded by Dutch colonists in 1626. However, when the English took control of the area, it was renamed New York in honor of the Duke of York, who was granted the lands by his brother, King Charles II. The name New York has remained ever since.',
 'source_documents': [Document(page_content="Etymology\nSee also: Nicknames of New York City\nIn 1664, New York was named in honor of the Duke of York (later King James\xa0II of England).[34] James's elder brother, King Charles\xa0II, appointed the Duke as proprietor of the former territory of New Netherland, including the city of New Amsterdam, when the Kingdom of England seized it from Dutch control.[35]", metadata={'source': 'https://en.wikipedia.org/wiki/New_York_City', 'title': 'New York City - Wikipedia', 'langua